# Analytics Vidhya - LTFS Data Science FinHack 2 - 0.6v
18-01-2020 12:00 AM to 26-01-2020 11:59 PM

## Problem Statement

LTFS receives a lot of requests for its various finance offerings that include housing loan, two-wheeler loan, real estate financing and micro loans. The number of applications received is something that varies a lot with season. Going through these applications is a manual process and is tedious. Accurately forecasting the number of cases received can help with resource and manpower management resulting into quick response on applications and more efficient processing.

You have been appointed with the task of forecasting daily cases for next 3 months for 2 different business segments at the country level keeping in consideration the following major Indian festivals (inclusive but not exhaustive list): Diwali, Dussehra, Ganesh Chaturthi, Navratri, Holi etc. (You are free to use any publicly available open source external datasets). Some other examples could be:

Weather
Macroeconomic variables
 Note that the external dataset must belong to a reliable source.

## Importing Packages

In [251]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
import re
import seaborn as sns
import matplotlib.pyplot as plt
from calendar import monthrange

### Basic Setups

#### Setting Working Directory

In [252]:
print('Current working Directory',os.getcwd())#Find out your current working directory
path = 'D:\\PrivateProjects\\LTFS\\' #setting path variable
os.chdir(path) # Changing to path's wk dir
print('Changed Working Directory',os.getcwd())#Find out your (changed) current working directory

Current working Directory D:\PrivateProjects\LTFS
Changed Working Directory D:\PrivateProjects\LTFS


#### Increase the cell size

In [253]:
# To increase the size of the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))  # To change the cell size

### Setting Display size
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 300)

#### User defined functions

In [254]:
#From fastai's module
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)
        
        
def printColumnInfo(df):
    template="%-8s %-30s %s"
    print(template % ("Type", "Column Name", "Example Value"))
    print("-"*53)
    for c in df.columns:
        print(template % (df[c].dtype, c, df[c].iloc[1]) )
        
#To get over all idea about the columns in dataset like count of unique values, type, % of missing values

def describe_df(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Percentage of missing values', 'Percentage of values in the biggest category', 'type'])
    stats_df.sort_values('Percentage of missing values', ascending=False)
    return stats_df

### Importing dataset

In [255]:
#Importing train csv file
df_raw_train_path =  (path +'data\\'+'train.csv') # Setting path for respective file
df_raw_train = pd.read_csv(df_raw_train_path) # importing train file
#del df_raw_train_path # deleting the path variable


#Importing test csv file
df_raw_test_path =  (path +'data\\'+'test.csv') # Setting path for respective file
df_raw_test = pd.read_csv(df_raw_test_path) # importing test file
#del df_raw_test_path # deleting the path variable

#### BACKUPS

In [256]:
df_raw_train_backup =    df_raw_train.copy()
df_raw_test_backup =    df_raw_test.copy()

### Analysis & Insights gaining

#### Looking through the files

##### Training data

In [257]:
print('Training Data shape',df_raw_train.shape,'\n')
printColumnInfo(df_raw_train)
df_raw_train.head()

Training Data shape (80402, 6) 

Type     Column Name                    Example Value
-----------------------------------------------------
object   application_date               2017-04-03
int64    segment                        1
float64  branch_id                      1.0
object   state                          WEST BENGAL
object   zone                           EAST
float64  case_count                     5.0


,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
1,2017-04-03,1,1.0,WEST BENGAL,EAST,5.0
2,2017-04-04,1,1.0,WEST BENGAL,EAST,4.0
3,2017-04-05,1,1.0,WEST BENGAL,EAST,113.0
4,2017-04-07,1,1.0,WEST BENGAL,EAST,76.0


##### Testing data

In [258]:
print('Testing Data shape',df_raw_test.shape,'\n')
printColumnInfo(df_raw_test)
df_raw_test.head()

Testing Data shape (180, 3) 

Type     Column Name                    Example Value
-----------------------------------------------------
int64    id                             2
object   application_date               2019-07-07
int64    segment                        1


,id,application_date,segment
0,1,2019-07-06,1
1,2,2019-07-07,1
2,3,2019-07-08,1
3,4,2019-07-09,1
4,5,2019-07-10,1


In [259]:
#Minimum & Maximum of dates for each segment
print(df_raw_train.groupby(['segment']).agg({'application_date': [np.min,np.max]}).reset_index())

  segment application_date            
                      amin        amax
0       1       2017-04-01  2019-07-05
1       2       2017-04-01  2019-07-23


In [260]:
df_raw_train.application_date = pd.to_datetime(df_raw_train.application_date)
df_raw_test.application_date = pd.to_datetime(df_raw_test.application_date)

In [261]:
#Lets seperate the training data based on segment
df_raw_train_seg1 = df_raw_train[df_raw_train.segment == 1]
df_raw_train_seg2 = df_raw_train[df_raw_train.segment == 2]

In [262]:
# Lets see if we have any missing values
print(df_raw_train_seg1.isna().sum())
print(df_raw_train_seg2.isna().sum())

application_date    0
segment             0
branch_id           0
state               0
zone                0
case_count          0
dtype: int64
application_date        0
segment                 0
branch_id           13504
state                   0
zone                13504
case_count              0
dtype: int64


We have missing values in branch_id & in zone for segment 2 df alone, anyway for now since our testing data is at country level lets groupby using application date

In [263]:
df_raw_train_seg1 = df_raw_train_seg1.groupby('application_date')['case_count'].sum().reset_index()
df_raw_train_seg2 = df_raw_train_seg2.groupby('application_date')['case_count'].sum().reset_index()

In [264]:
df_raw_train_seg1.set_index('application_date', inplace=True)
df_raw_train_seg2.set_index('application_date', inplace=True)

In [265]:
print('Days in Segment 1',df_raw_train_seg1.index.max() - df_raw_train_seg1.index.min())
print('Unique Dates in Segment 1',df_raw_train_seg1.index.nunique())
print('Days in Segment 2',df_raw_train_seg2.index.max() - df_raw_train_seg2.index.min())
print('Unique Dates in Segment 2',df_raw_train_seg2.index.nunique())

Days in Segment 1 825 days 00:00:00
Unique Dates in Segment 1 806
Days in Segment 2 843 days 00:00:00
Unique Dates in Segment 2 844


We can see that we have missing dates, lets fill those first

###### Missing dates for each segment

In [266]:
#Missing dates in segment 1
pd.date_range(start = df_raw_train_seg1.index.min(), end = df_raw_train_seg1.index.max() ).difference(df_raw_train_seg1.index)

DatetimeIndex(['2017-04-02', '2017-04-06', '2017-04-08', '2017-04-09',
               '2017-04-10', '2017-04-11', '2017-05-06', '2017-07-03',
               '2017-07-04', '2017-07-08', '2017-07-09', '2017-08-02',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-06',
               '2018-01-07', '2018-02-04', '2018-03-02', '2018-03-04'],
              dtype='datetime64[ns]', freq=None)

In [267]:
#Missing dates in segment 2
pd.date_range(start = df_raw_train_seg2.index.min(), end = df_raw_train_seg2.index.max() ).difference(df_raw_train_seg2.index)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

###### Insights :  
-  We have totally 5 columns in training (excluding the target column 'case_count') but only 2 columns in test (excluding the id column)
        - Missing columns in test are [branch_id, state, zone]
   This means we have to train the model at country level & not at branch_id level
- The training data for segment 1 is from 2017-04-01 to 2019-07-05,825 days, we have missing dates
- The training data for segment 2 is from 2017-04-01 to 2019-07-23,843 days

###### Lets include the missing dates for each sement

In [268]:
# resetting the index (application_date)
df_raw_train_seg1.reset_index(inplace=True)
df_raw_train_seg2.reset_index(inplace=True)

In [269]:
seg1_dates = pd.DataFrame(pd.date_range(start = df_raw_train_seg1.application_date.min(), end = df_raw_train_seg1.application_date.max() ))
seg1_dates.columns = ['application_date']

seg2_dates = pd.DataFrame(pd.date_range(start = df_raw_train_seg2.application_date.min(), end = df_raw_train_seg2.application_date.max() ))
seg2_dates.columns = ['application_date']

In [270]:
df_raw_train_seg1 = pd.merge(seg1_dates, df_raw_train_seg1,how = 'left', on ='application_date')
df_raw_train_seg2 = pd.merge(seg2_dates, df_raw_train_seg2,how = 'left', on ='application_date')

In [271]:
#Check if we have any more missing dates
df_raw_train_seg1.set_index('application_date', inplace=True)
df_raw_train_seg2.set_index('application_date', inplace=True)

print(pd.date_range(start = min(df_raw_train_seg1.index), end = max(df_raw_train_seg1.index) ).difference(df_raw_train_seg1.index))
print(pd.date_range(start = min(df_raw_train_seg2.index), end = max(df_raw_train_seg2.index) ).difference(df_raw_train_seg2.index))

DatetimeIndex([], dtype='datetime64[ns]', freq='D')
DatetimeIndex([], dtype='datetime64[ns]', freq='D')


In [272]:
#Filling the missing values using linear method
df_raw_train_seg1['case_count'].interpolate(method='linear',inplace = True)

#### Now its time to make the validation set in accordance with the test set to make sure the model validation is done right before predicting the model on the original test set

###### How we can create the validation set?

In [273]:
df_raw_test_seg1 = df_raw_test[df_raw_test.segment == 1]
df_raw_test_seg2 = df_raw_test[df_raw_test.segment == 2]

In [280]:
print('Segment 1 train data from ', df_raw_train_seg1.index.min(),'to',df_raw_train_seg1.index.max())
print('Segment 1 test data from ', df_raw_test_seg1.application_date.min(),'to',df_raw_test_seg1.application_date.max())
print('Segment 1 test number of days', df_raw_test_seg1.application_date.min()-df_raw_test_seg1.application_date.max())

Segment 1 train data from  2017-04-01 00:00:00 to 2019-07-05 00:00:00
Segment 1 test data from  2019-07-06 00:00:00 to 2019-09-30 00:00:00
Segment 1 test number of days -86 days +00:00:00


For segment 1 we have data up to 5th of July and we need to predict for next 86 days 

In [281]:
print('Segment 2 train data from ', df_raw_train_seg2.index.min(),'to',df_raw_train_seg2.index.max())
print('Segment 2 test data from ', df_raw_test_seg2.application_date.min(),'to',df_raw_test_seg2.application_date.max())
print('Segment 2 test number of days', df_raw_test_seg2.application_date.min()-df_raw_test_seg2.application_date.max())

Segment 2 train data from  2017-04-01 00:00:00 to 2019-07-23 00:00:00
Segment 2 test data from  2019-07-24 00:00:00 to 2019-10-24 00:00:00
Segment 2 test number of days -92 days +00:00:00


For segment 2 we have data up to 23rd of July and we need to predict for next 92 days 

In [283]:
df_raw_train_seg1

,case_count
application_date,
2017-04-01,299.0
2017-04-02,170.5
2017-04-03,42.0
2017-04-04,23.0
2017-04-05,1530.0
2017-04-06,1435.5
2017-04-07,1341.0
2017-04-08,1366.4
2017-04-09,1391.8
